In [6]:
from datetime import datetime, timezone, timedelta
import pandas as pd

from database.market import Market
from database.sec import SEC
from processor.processor import Processor as p
from modeler_strats.universal_modeler import UniversalModeler
from fund.fund import Fund

from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio

In [12]:
portfolio = APortfolio(pricer_list.QUARTERLY_STOCK_FINANCIAL
                          ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolios = []
portfolios.append(portfolio)
fund = Fund(portfolios,datetime(2023,1,1),datetime.now(),datetime.now())
fund.initialize_portfolios()

In [27]:
year = 2023

In [19]:
market = Market()
market.cloud_connect()
sp500 = market.retrieve("sp500").rename(columns={"Symbol":"ticker"})
market.disconnect()
modeler_strat = UniversalModeler()

In [32]:
for portfolio in fund.portfolios:
    try:
        pricer_class = portfolio.pricer_class
        tickers = sp500["ticker"].unique() if pricer_class.asset_class.value == "stocks" else ["BTC"]
        training_sets = []
        sec.connect()
        market.cloud_connect()
        for ticker in tickers:
            try:
                cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
                prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
                filings = sec.retrieve_filing_data(cik)
                prices = p.column_date_processing(prices)
                filings = p.column_date_processing(filings)
                ticker_data = pricer_class.training_set(ticker,prices,filings,True)
                training_sets.append(ticker_data)
            except Exception as e:
                print(ticker,str(e))
                continue
        data = pd.concat(training_sets)
        training_data = data.dropna().copy().sort_values(["year","week"])
        market.disconnect()
        sec.disconnect()
    except Exception as e:
        print(str(e))
        continue

AJG "['weightedaveragenumberofdilutedsharesoutstanding'] not in index"
T "['stockholdersequity'] not in index"
BKR "['weightedaveragenumberofsharesoutstandingbasic'] not in index"
CPT "['incometaxexpensebenefit', 'retainedearningsaccumulateddeficit', 'propertyplantandequipmentnet'] not in index"
CI "['weightedaveragenumberofsharesoutstandingbasic'] not in index"
CSX "['stockholdersequity'] not in index"
HIG "['cashandcashequivalentsatcarryingvalue'] not in index"
HSY "['earningspersharebasic', 'earningspersharediluted'] not in index"
TAP "['entitycommonstocksharesoutstanding'] not in index"
UDR "['retainedearningsaccumulateddeficit', 'propertyplantandequipmentnet'] not in index"
UAA "['entitycommonstocksharesoutstanding'] not in index"
ALL "['cashandcashequivalentsatcarryingvalue'] not in index"
AXP "['cashandcashequivalentsatcarryingvalue'] not in index"
BIO "['entitycommonstocksharesoutstanding'] not in index"
EXPE "['entitycommonstocksharesoutstanding'] not in index"
FFIV "['stockho

In [33]:
training_data = data.copy()

In [34]:
training_data.columns

Index(['assets', 'liabilitiesandstockholdersequity', 'incometaxexpensebenefit',
       'retainedearningsaccumulateddeficit',
       'accumulatedothercomprehensiveincomelossnetoftax',
       'earningspersharebasic', 'earningspersharediluted',
       'propertyplantandequipmentnet', 'cashandcashequivalentsatcarryingvalue',
       'entitycommonstocksharesoutstanding',
       'weightedaveragenumberofdilutedsharesoutstanding',
       'weightedaveragenumberofsharesoutstandingbasic', 'stockholdersequity',
       'year', 'quarter', 'ticker', 'adjclose'],
      dtype='object')

In [35]:
# making predictions
if pricer_class.isai:
    pricer_class.db.cloud_connect()
    pricer_class.db.drop("predictions")
    prediction_set = training_data[training_data["year"]==year-1].reset_index(drop=True)
    models = pricer_class.db.retrieve("models")
    stuff = modeler_strat.recommend(models,prediction_set,pricer_class.factors)
    stuff = stuff.rename(columns={"prediction":f"price_prediction"})
    stuff = pricer_class.sim_processor(stuff)
    relevant_columns = list(set(list(stuff.columns)) - set(pricer_class.factors))
    pricer_class.db.store("predictions",stuff[relevant_columns])
    pricer_class.db.disconnect()

In [37]:
stuff[relevant_columns].to_csv("financial_predictions.csv")